In [ ]:
!pip install fastapi uvicorn pyngrok diffusers transformers accelerate safetensors torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.1 MB/s eta 0:00:00


In [ ]:
import torch
from diffusers import DiffusionPipeline
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.responses import FileResponse
from PIL import Image
import uvicorn
import os
import nest_asyncio

In [ ]:
# Load the model
model_id = "cloudqi/cqi_text_to_image_pt_v0"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.to("cuda")  # Use GPU for faster generation (if available)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.32.2",
  "_name_or_path": "cloudqi/cqi_text_to_image_pt_v0",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
# Define FastAPI app
app = FastAPI()

# Request model
class PromptRequest(BaseModel):
    prompt: str

# Endpoint to generate image
@app.post("/generate")
def generate_image(request: PromptRequest):
    image = pipe(prompt=request.prompt).images[0]
    image_path = "generated_image.png"
    image.save(image_path)
    return FileResponse(image_path, media_type="image/png")

# Fix event loop issue in Colab
nest_asyncio.apply()

# Run the API in Colab
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [333]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     34.118.206.186:0 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     2409:40e6:29:e2a3:9ec:513a:93d1:d249:0 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [333]


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Set your ngrok auth token (Run only once)
!ngrok config add-authtoken NGROK_AUTH_TOKEN

# Start ngrok
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: https://327c-34-169-252-237.ngrok-free.app
